In [1]:
import pandas as pd
import os
import seaborn as sns
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt
import category_encoders as ce

Populating the interactive namespace from numpy and matplotlib


In [2]:
for dirname, _, filenames in os.walk('../../Datas/ashre-energy/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../../Datas/ashre-energy/test_df.pkl
../../Datas/ashre-energy/FE_train_V1.pkl
../../Datas/ashre-energy/FE_test_V1.pkl
../../Datas/ashre-energy/test.pkl
../../Datas/ashre-energy/train.pkl
../../Datas/ashre-energy/building_metadata.pkl
../../Datas/ashre-energy/weather_train.pkl
../../Datas/ashre-energy/test.csv
../../Datas/ashre-energy/sample_submission.csv
../../Datas/ashre-energy/train.csv
../../Datas/ashre-energy/weather_test.pkl
../../Datas/ashre-energy/building_metadata.csv
../../Datas/ashre-energy/weather_test.csv
../../Datas/ashre-energy/weather_train.csv
../../Datas/ashre-energy/submission.csv
../../Datas/ashre-energy/.gitignore
../../Datas/ashre-energy/train_df.pkl
../../Datas/ashre-energy/.ipynb_checkpoints/.gitignore-checkpoint


In [3]:
train = pd.read_pickle('../../Datas/ashre-energy/train_df.pkl')
test = pd.read_pickle('../../Datas/ashre-energy/test_df.pkl')

In [4]:
#Onehot encode and drop columns
def onehotEncode(df,dic):
    for col,drop in dic.items():
        df = pd.concat([df,pd.get_dummies(df[col])],axis=1)
        if drop:
            df.drop(col,axis=1,inplace=True)
    return df

In [5]:
def meanEncode(df,dic):
#     mean_list = []
    for col,drop in dic.items():
        mean = df[col].mean()
#         mean_list.append(mean)
        df[col+'_mean_encode'] = df[col]-mean
        if drop:
            df.drop(col,axis=1,inplace=True)
    return df#, mean_list

# def meanEncode(df,dic,mean_list):
#     for col,drop in dic.items():
#         mean = mean_list.pop(0)
#         df[col+'_mean_encode'] = df[col]-mean
#         if drop:
#             df.drop(col,axis=1,inplace=True)
#     return df

In [5]:
def seeHis(df,col):
    sns.distplot(df[col].dropna(),label=col)

In [ ]:
to_onehot = {'primary_use':True,'meter':True}
to_mean_encode = {'year_built':True}

In [ ]:
#Make workday
train['workday'] = train['DT_day_week'].isin([0,6]).astype(np.int8)

In [ ]:
#Group wind in to catergory
train['new_wind_dir'] = pd.cut(train['wind_direction'],bins=8)
train.drop('wind_direction',axis=1,inplace=True)
to_onehot['new_wind_dir'] = True

In [ ]:
train['floor_count'].replace(0,1,inplace=True)
train['avg_floor_area'] = train['square_feet']/train['floor_count']

In [ ]:
train['year_built'].replace(-999,np.nan,inplace=True)

In [ ]:
train = onehotEncode(train,to_onehot)
train = meanEncode(train,to_mean_encode)

In [ ]:
train.columns.to_list()

In [3]:
train.to_pickle('../../Datas/ashre-energy/FE_train_V1.pkl')

NameError: name 'train' is not defined

In [6]:
def pipeline(train_df,test_df):
    train_len = train_df.shape[0]
    df = pd.concat([train_df,test_df],sort=False)
    del train_df, test_df
    
    to_onehot = {'primary_use':True,'meter':True}
    to_mean_encode = {'year_built':True}
    
    #Make workday
    df['workday'] = df['DT_day_week'].isin([0,6]).astype(np.int8)

    #Group wind in to catergory
    df['new_wind_dir'] = pd.cut(df['wind_direction'],bins=8)
    df.drop('wind_direction',axis=1,inplace=True)
    to_onehot['new_wind_dir'] = True
    
    #Make average floor
    df['floor_count'].replace(0,1,inplace=True)
    df['avg_floor_area'] = df['square_feet']/df['floor_count']
    
    #Clean year_built
    df['year_built'].replace(-999,np.nan,inplace=True)
    df = onehotEncode(df,to_onehot)
    df = meanEncode(df,to_mean_encode)
    df.fillna(df.mean(),inplace=True)
    
    return df.head(train_len),df.tail(train_len)

In [7]:
train_df = pd.read_pickle('../../Datas/ashre-energy/train_df.pkl')
test_df = pd.read_pickle('../../Datas/ashre-energy/test_df.pkl')

In [8]:
train_df, test_df = pipeline(train_df,test_df)

MemoryError: Unable to allocate array with shape (8, 61913700) and data type float64

In [9]:
mean = train_df.mean()

/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


In [11]:
train_df.fillna(mean,inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [15]:
train_df.isna().sum()

DT_D                             0
DT_M                             0
DT_W                             0
DT_day_month                     0
DT_day_week                      0
DT_hour                          0
DT_week_month                    0
air_temperature              96658
building_id                      0
cloud_coverage             8825365
dew_temperature             100140
floor_count                      0
meter_reading                    0
precip_depth_1_hr          3749023
row_id                    20216100
sea_level_pressure         1231669
site_id                          0
square_feet                      0
timestamp                        0
wind_speed                  143676
workday                          0
avg_floor_area                   0
(-0.36, 45.0]                    0
(45.0, 90.0]                     0
(90.0, 135.0]                    0
(135.0, 180.0]                   0
(180.0, 225.0]                   0
(225.0, 270.0]                   0
(270.0, 315.0]      

In [ ]:
train_df.to_pickle('../../Datas/ashre-energy/FE_train_V2_fillmean.pkl')
del train_df
test_df.to_pickle('../../Datas/ashre-energy/FE_test_V2_fillmean.pkl')
del test_df